In [23]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [25]:
import nltk
from nltk import pos_tag, word_tokenize
text=word_tokenize("Hello welcome to the world of learning Categoizing and Pos tagging with NLTK and Python")
nltk.pos_tag(text)

[('Hello', 'NNP'),
 ('welcome', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('world', 'NN'),
 ('of', 'IN'),
 ('learning', 'VBG'),
 ('Categoizing', 'NNP'),
 ('and', 'CC'),
 ('Pos', 'NNP'),
 ('tagging', 'VBG'),
 ('with', 'IN'),
 ('NLTK', 'NNP'),
 ('and', 'CC'),
 ('Python', 'NNP')]

In [26]:
#Printing only nouns from the text:
is_noun=lambda x:x[:3]=='NNP'
nouns=[word for(word,x) in pos_tag(text) if is_noun(x)]
print(nouns)

['Hello', 'Categoizing', 'Pos', 'NLTK', 'Python']


In [27]:
#Lets create the topic modelling of only nouns from the newspaper data

In [32]:
#Import Libraries
import os
import numpy as np
import pandas as pd
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation

In [35]:
# Getting the data from sklearn
from sklearn.datasets import fetch_20newsgroups

In [38]:
sub_category = ['comp.sys.mac.hardware', 'rec.autos', 'sci.space', 'misc.forsale','talk.politics.guns']

news_group=fetch_20newsgroups(categories=sub_category,remove=('headers','footers','quotes'))

In [47]:
docs_raw=news_group.data

In [48]:
#Cleaning the Data

In [49]:
df = pd.DataFrame(docs_raw)

In [50]:
df=df.rename(columns={0:'Raw_text'})
df.head()

,Raw_text
0,\n740 Turbo in UK was good for 124mph. Useful ...
1,I am looking for a working docking deck (deck ...
2,\n >In article <1993Apr19.020359.26996@sq.sq.c...
3,"A few posts back, somebody mentioned that the ..."
4,"\n\n\n\n\n\nIt still applies, except the astro..."


In [61]:
#Selecting only words
doc = df['Raw_text'].str.lower().str.replace('[^a-z ]','')

In [62]:
#Lets remove the stop words:
stop=set(stopwords.words('english'))
def sw(text):
    text=[word for word in text.split() if word not in stop]
    #joining the list of words with space seperator
    return " ".join(text)
doc_clean=doc.apply(sw)
doc_clean.head()

0    turbo uk good mph useful blowing away vw beetl...
1    looking working docking deck deck goes back of...
2    article aprsqsqcom msbsqsqcom mark brader mb t...
3    posts back somebody mentioned duo might crash ...
4    still applies except astronomy days long basel...
Name: Raw_text, dtype: object

In [63]:
type(doc_clean)

pandas.core.series.Series

In [64]:
# Now we will try to create the topic modelling only using nouns so that we have a bit more clear idea what topics are all about
# creating the user define function

In [65]:
def nouns(x):
    is_noun=lambda x:x[:3]=='NN'
    tokenized=word_tokenize(x)
    all_noun=[word for (word,x) in pos_tag(tokenized) if is_noun(x)]
    return " ".join(all_noun)

In [66]:
doc_finall=doc_clean.apply(nouns)

In [67]:
doc_finall.head()

0                 turbo mph beetle doubt car volvo car
1    deck deck jvc gxs tube video camera format ple...
2    article aprsqsqcom msbsqsqcom mark brader mb f...
3    somebody duo hasthe kind duo software whichwil...
4    astronomy nasas observatorys source rotation t...
Name: Raw_text, dtype: object

In [69]:
# Creating the LDA model again with new DTM which contain only Nouns

vec_noun = CountVectorizer(max_df=.3,min_df=10)
dtm_noun = vec_noun.fit_transform(doc_finall)
dtm_noun.shape

(2896, 1105)

In [70]:
# Creating the LDA model
lda_model_noun = LatentDirichletAllocation(n_components=5, max_iter=50)
# Fitting the model on dtm
lda_output_noun = lda_model_noun.fit_transform(dtm_noun)